In [56]:
# This function should open a data file in csv, and transform it into a usable format 
#def preprocess_unsupervised():     
import pandas as pd
import math
import random
from collections import defaultdict

# Convert CSV to dataframe

# Select from breast-cancer, mushroom, hypothyroid
filename = 'breast-cancer-dos.csv'
df = pd.read_csv(filename, header=None)

class1 = class2 = 0
df_len = len(df.columns) - 1
classes_list = df[df_len].unique()
num_classes = len(classes_list)
num_instances = len(df)
attributes_len = df_len - num_classes
labelled_data = df[df_len].copy(deep=True)

# This function should build an unsupervised NB model 
#def train_unsupervised():
for i in range(num_instances): 
    df.at[i, df_len-1] = random.uniform(0, 1)
    df.at[i, df_len] = 1 - df[df_len-1][i]
    class1 += df.at[i, df_len-1] 
    class2 += df.at[i, df_len]

# Generate attribute instance frequencies for both classes
reccurence_freq = [dict() for x in range(attributes_len)]
no_reccurence_freq = [dict() for x in range(attributes_len)]

for i in range(attributes_len):
    reccurence_freq[i] = defaultdict(int)
    for j in range(num_instances):
        reccurence_freq[i][df[i][j]] += df[df_len-1][j]

for i in range(attributes_len):
    no_reccurence_freq[i] = defaultdict(int)
    for j in range(num_instances):
        no_reccurence_freq[i][df[i][j]] += df[df_len][j]

for i in range(attributes_len):
    for key in reccurence_freq[i]:
        reccurence_freq[i][key] = reccurence_freq[i][key]/class1

for i in range(attributes_len):
    for key in no_reccurence_freq[i]:
        no_reccurence_freq[i][key] = no_reccurence_freq[i][key]/class2

# This function should predict the class distribution for a set of instances, based on a trained model
#def predict_unsupervised():

# N.B. I realise this is not the most efficient algorithm -- I was having trouble getting it to work since I'm not very familiar
# with Python, so I designed the algorithm to replicate each step of the process as a "visualised" sanity check of sorts.

prediction = [defaultdict(int) for x in range(num_instances)]
prediction2 = [defaultdict(int) for x in range(num_instances)]
num_iterations = 4
true_positive = true_negative = false_negative = false_positive = 0

for iteration in range(num_iterations):
    if iteration == 0:
        for i in range(num_instances):
            for j in range(attributes_len):
                prediction[i][df[j][i]] = reccurence_freq[j][df[j][i]]
                prediction[i]['temp'] = 1

        for i in range(num_instances):
            for j in range(attributes_len):
                prediction2[i][df[j][i]] = no_reccurence_freq[j][df[j][i]]
                prediction2[i]['temp'] = 1

    for i in range(num_instances):
        for j in range(attributes_len):
            prediction[i]['temp'] *= prediction[i][df[j][i]]
        prediction[i]['pred'] = 1
        prediction[i]['pred'] *= class1/num_instances
        prediction[i]['pred'] *= prediction[i]['temp']

    for i in range(num_instances):
        for j in range(attributes_len):
            prediction2[i]['temp'] *= prediction2[i][df[j][i]]
        prediction2[i]['pred'] = 1
        prediction2[i]['pred'] *= class2/num_instances
        prediction2[i]['pred'] *= prediction2[i]['temp']

    for i in range(num_instances):
        prediction[i]['newclass'] = prediction[i]['pred']/(prediction[i]['pred']+prediction2[i]['pred'])
        prediction2[i]['newclass'] = prediction2[i]['pred']/(prediction[i]['pred']+prediction2[i]['pred'])

# This function should evaluate a set of predictions, in an unsupervised manner
#def evaluate_unsupervised():
for i in range(num_instances):        
    if(prediction[i]['newclass'] >= prediction2[i]['newclass']):
        if(labelled_data[i]==str(classes_list[0])):
            true_positive += 1
        else:
            false_positive += 1
    elif(prediction[i]['newclass'] < prediction2[i]['newclass']):
        if(labelled_data[i]==str(classes_list[1])):
            true_negative += 1
        else:
            false_negative += 1

print("Accuracy: " + str(round((true_positive+true_negative)*100/num_instances, 2)) + "%\n")

print("CONFUSION MATRIX\nTrue Positive:  " + str(true_positive) + "     True Negative: " + str(true_negative) 
      + "\nFalse Negative: " + str(false_negative) + "     False Positive: " + str(false_positive))

Accuracy: 52.45%

CONFUSION MATRIX
True Positive:  46     True Negative: 104
False Negative: 39     False Positive: 97
